In [1]:
import pandas as pd
import os
# Loading each dataset manually
data1 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"))
data2 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"))
data3 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Friday-WorkingHours-Morning.pcap_ISCX.csv"))
data4 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Monday-WorkingHours.pcap_ISCX.csv"))
data5 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"))
data6 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"))
data7 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Tuesday-WorkingHours.pcap_ISCX.csv"))
data8 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Wednesday-workingHours.pcap_ISCX.csv"))

# Combine all dataframes into one
data_list = [data1, data2, data3, data4, data5, data6, data7, data8]
df = pd.concat(data_list, ignore_index=True)
print(df.columns.tolist())

[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag 

In [3]:
import numpy as np
df.columns = df.columns.str.strip()
features = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
            'Fwd Packet Length Mean', 'Bwd Packet Length Mean',
            'Flow Bytes/s', 'Fwd Packets/s', 'Packet Length Std']
X = df[features].fillna(0)

# Remove infinite values
X.replace([np.inf, -np.inf], 0, inplace=True)
df['Label'] = df['Label'].astype(str).apply(lambda x: 0 if 'Benign' in x else 1)
y = df['Label']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Assume X and y are already defined above this
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, predictions) * 100:.2f}%")

joblib.dump(model, 'intrusion_detection_model.pkl')

Model Accuracy: 100.00%


['intrusion_detection_model.pkl']

In [7]:
import joblib

# Load the pre-trained model
model = joblib.load('intrusion_detection_model.pkl')

In [33]:
import scapy.all as scapy
import numpy as np

# Initialize flow variables globally
flow_start_time = None
flow_end_time = None
flow_source_ip = None
flow_dest_ip = None
fwd_packets = 0
bwd_packets = 0
fwd_pkt_len = []
bwd_pkt_len = []
flow_bytes = 0
flow_packets = []

def extract_features(packet):
    global flow_start_time, flow_end_time, flow_source_ip, flow_dest_ip
    global fwd_packets, bwd_packets, fwd_pkt_len, bwd_pkt_len, flow_bytes, flow_packets

    try:
        # Only process IP packets
        if packet.haslayer(scapy.IP):
            print(f"Processing packet: {packet[scapy.IP].src} -> {packet[scapy.IP].dst}")  # Debugging print

            # Set source and destination IP for the flow (if not already set)
            if flow_source_ip is None and flow_dest_ip is None:
                flow_source_ip = packet[scapy.IP].src
                flow_dest_ip = packet[scapy.IP].dst
                print(f"Flow started: {flow_source_ip} -> {flow_dest_ip}")  # Debugging print

            # Track flow duration
            if flow_start_time is None:
                flow_start_time = packet.time
                print(f"Flow start time: {flow_start_time}")  # Debugging print
            flow_end_time = packet.time

            # Count forward and backward packets and track their lengths
            if packet[scapy.IP].src == flow_source_ip and packet[scapy.IP].dst == flow_dest_ip:
                fwd_packets += 1
                fwd_pkt_len.append(len(packet))
                print(f"Forward packet: {len(packet)} bytes")  # Debugging print
            elif packet[scapy.IP].src == flow_dest_ip and packet[scapy.IP].dst == flow_source_ip:
                bwd_packets += 1
                bwd_pkt_len.append(len(packet))
                print(f"Backward packet: {len(packet)} bytes")  # Debugging print

            # Count total flow bytes
            flow_bytes += len(packet)

            # Store the packet for further analysis (optional)
            flow_packets.append(packet)
            # Calculate features when flow duration is greater than 0
            if flow_end_time - flow_start_time > 0:  # flow duration is non-zero
                flow_duration = flow_end_time - flow_start_time
                fwd_pkt_len_mean = np.mean(fwd_pkt_len) if fwd_pkt_len else 0
                bwd_pkt_len_mean = np.mean(bwd_pkt_len) if bwd_pkt_len else 0
                flow_bytes_per_second = flow_bytes / flow_duration if flow_duration > 0 else 0
                fwd_packets_per_second = fwd_packets / flow_duration if flow_duration > 0 else 0
                pkt_len_std = np.std([len(pkt) for pkt in flow_packets]) if flow_packets else 0

                # Debugging print to check the extracted features
                print(f"Extracted features: [flow_duration: {flow_duration}, "
                      f"fwd_packets: {fwd_packets}, bwd_packets: {bwd_packets}, "
                      f"fwd_pkt_len_mean: {fwd_pkt_len_mean}, bwd_pkt_len_mean: {bwd_pkt_len_mean}, "
                      f"flow_bytes_per_second: {flow_bytes_per_second}, "
                      f"fwd_packets_per_second: {fwd_packets_per_second}, pkt_len_std: {pkt_len_std}]")

                return [flow_duration, fwd_packets, bwd_packets, fwd_pkt_len_mean, bwd_pkt_len_mean,
                        flow_bytes_per_second, fwd_packets_per_second, pkt_len_std]            
            else:
                print("Flow duration is zero or negative, skipping feature extraction.")  # Debugging print
        else:
            print("Packet is not an IP packet.")  # Debugging print
    except Exception as e:
        print(f"Error extracting features: {e}")
        return None

# Example function to call extract_features and detect intrusion
def packet_callback(packet):
    features = extract_features(packet)
    if features:
        print(f"Features extracted: {features}")  # Debugging print
    else:
        print("No features extracted.")  # Debugging print

scapy.sniff(prn=packet_callback, store=0, count=100) 
           

Processing packet: 192.168.0.87 -> 255.255.255.255
Flow started: 192.168.0.87 -> 255.255.255.255
Flow start time: 1744603443.676001
Forward packet: 241 bytes
Flow duration is zero or negative, skipping feature extraction.
No features extracted.
Packet is not an IP packet.
No features extracted.
Packet is not an IP packet.
No features extracted.
Processing packet: 192.168.0.152 -> 3.166.96.120
Extracted features: [flow_duration: 2.4860148429870605, fwd_packets: 1, bwd_packets: 0, fwd_pkt_len_mean: 241.0, bwd_pkt_len_mean: 0, flow_bytes_per_second: 133.14482048799368, fwd_packets_per_second: 0.40225021295466373, pkt_len_std: 75.5]
Features extracted: [2.4860148429870605, 1, 0, 241.0, 0, 133.14482048799368, 0.40225021295466373, 75.5]
Processing packet: 192.168.0.152 -> 3.166.96.120
Extracted features: [flow_duration: 2.48685884475708, fwd_packets: 1, bwd_packets: 0, fwd_pkt_len_mean: 241.0, bwd_pkt_len_mean: 0, flow_bytes_per_second: 159.63913707325014, fwd_packets_per_second: 0.402113695

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [34]:
intrusion_count = 0  # Track the number of intrusions

def detect_intrusion(packet):
    global intrusion_count
    max_intrusions = 20  # Stop after detecting 3 intrusions

    # Continue sniffing if we haven't reached the maximum intrusion threshold
    if intrusion_count >= max_intrusions:
        return True  # Return True to stop sniffing

    features = extract_features(packet)
    if features:
        # Convert features to DataFrame for the model
        feature_df = pd.DataFrame([features], columns=['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
                                                      'Fwd Packet Length Mean', 'Bwd Packet Length Mean',
                                                      'Flow Bytes/s', 'Fwd Packets/s', 'Packet Length Std'])
        
        # Fill missing values with 0 and handle infinite values
        feature_df = feature_df.fillna(0)
        feature_df.replace([np.inf, -np.inf], 0, inplace=True)
        
        # Predict using the model
        prediction = model.predict(feature_df)
        
        # Print result (0 for benign, 1 for attack)
        if prediction == 1:
            intrusion_count += 1  # Increment intrusion count
            print(f"Intrusion detected! Total intrusions detected: {intrusion_count}")
            
            # Stop sniffing if we have detected the maximum number of intrusions
            if intrusion_count >= max_intrusions:
                print("Max intrusions detected, stopping sniffing.")
                return True  # Return True to stop sniffing
        else:
            print("Benign traffic detected.")
    
    return False  # Continue sniffing if not reached max intrusions

# Start sniffing and detect intrusions, stop if the specified number of intrusions is detected
scapy.sniff(prn=detect_intrusion, store=0, stop_filter=detect_intrusion)

Packet is not an IP packet.
False
Packet is not an IP packet.
Packet is not an IP packet.
False
Packet is not an IP packet.
Packet is not an IP packet.
False
Packet is not an IP packet.
Packet is not an IP packet.
False
Packet is not an IP packet.
Packet is not an IP packet.
False
Packet is not an IP packet.
Packet is not an IP packet.
False
Packet is not an IP packet.
Packet is not an IP packet.
False
Packet is not an IP packet.
Processing packet: 192.168.0.87 -> 255.255.255.255
Forward packet: 241 bytes
Extracted features: [flow_duration: 21.398260831832886, fwd_packets: 8, bwd_packets: 0, fwd_pkt_len_mean: 241.0, bwd_pkt_len_mean: 0, flow_bytes_per_second: 147.95594954568153, fwd_packets_per_second: 0.373862159306839, pkt_len_std: 78.91746304548171]
Intrusion detected! Total intrusions detected: 1
False
Processing packet: 192.168.0.87 -> 255.255.255.255
Forward packet: 241 bytes
Extracted features: [flow_duration: 21.398260831832886, fwd_packets: 9, bwd_packets: 0, fwd_pkt_len_mean:

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [95]:
normal_html = """
<html>
  <head>
    <title>Welcome to My Website</title>
    <style>
      body {
        font-family: Arial, sans-serif;
        background-color: #f4f4f4;
        color: #333;
        margin: 0;
        padding: 0;
      }
      header {
        background-color: #2c3e50;
        color: white;
        padding: 20px;
        text-align: center;
      }
      nav {
        background-color: #34495e;
        padding: 10px;
        text-align: center;
      }
      nav a {
        color: white;
        text-decoration: none;
        margin: 0 15px;
        font-weight: bold;
      }
      .content {
        padding: 20px;
      }
      footer {
        background-color: #2c3e50;
        color: white;
        text-align: center;
        padding: 15px;
        position: fixed;
        width: 100%;
        bottom: 0;
      }
    </style>
  </head>
  <body>
    <header>
      <h1>My Secure Website</h1>
      <p>Welcome! Your connection is secure.</p>
    </header>

    <nav>
      <a href="#">Home</a>
      <a href="#">About</a>
      <a href="#">Services</a>
      <a href="#">Contact</a>
    </nav>

    <div class="content">
      <h2>About This Site</h2>
      <p>This is a demonstration website used in a cybersecurity project.</p>
      <p>It simulates the appearance and functionality of a typical secure website that is not under attack.</p>

      <h3>Why Cybersecurity Matters</h3>
      <p>Maintaining strong security practices protects user data, prevents attacks, and builds trust.</p>
    </div>

    <footer>
      &copy; 2025 Secure Web Inc. | All Rights Reserved
    </footer>
  </body>
</html>
"""

with open("normal.html", "w") as f:
    f.write(normal_html)

print("✅ Normal website created successfully.")


✅ Normal website created successfully.


In [97]:
from IPython.display import display, HTML

display(HTML("""
    <button onclick="window.open('normal.html')">🟢 Open Normal Website</button>
"""))


In [99]:
hacked_html = """
<html>
  <head>
    <title>H4CK3D | SYSTEM BREACHED</title>
    <style>
      body {
        background-color: black;
        color: red;
        font-family: 'Courier New', monospace;
        text-align: center;
        margin: 0;
        padding: 0;
        animation: blink 1s step-start infinite;
      }
      h1 {
        font-size: 60px;
        margin-top: 50px;
        text-shadow: 0 0 10px red;
      }
      p {
        font-size: 22px;
        color: white;
        margin-top: 20px;
      }
      .hacker {
        font-size: 28px;
        color: red;
        margin-top: 40px;
        animation: flicker 1.5s infinite alternate;
      }
      @keyframes blink {
        50% { opacity: 0.5; }
      }
      @keyframes flicker {
        0% { opacity: 1; }
        100% { opacity: 0.2; }
      }
    </style>
  </head>
  <body>
    <h1>⚠️ YOU'VE BEEN HACKED ⚠️</h1>
    <p>All your data has been encrypted. System breached by <strong>DarkNightX</strong>.</p>
    <div class="hacker">
      <p># Owned by DarkNet Crew</p>
      <p>We are everywhere. We are watching.</p>
    </div>
    <audio autoplay loop>
      <source src="https://www.myinstants.com/media/sounds/trololo.mp3" type="audio/mpeg">
    </audio>
  </body>
</html>
"""

with open("hacked.html", "w") as f:
    f.write(hacked_html)

print("🚨 Hacked website created.")


🚨 Hacked website created.


In [101]:
from IPython.display import display, HTML

display(HTML("""
    <button onclick="window.open('hacked.html')">🔴 View Hacked Website</button>
"""))
